In [161]:
import pandas as pd
import numpy as np

In [162]:
random_seed = 33
np.random.seed(random_seed)

In [163]:
users = pd.read_csv('data/users.csv', encoding='iso-8859-1')
engagement = pd.read_csv('data/user_engagement.csv', encoding='iso-8859-1')

In [164]:
users.shape, engagement.shape

((12000, 10), (207917, 3))

In [165]:
users.info(), engagement.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 12000 entries, 0 to 11999
Data columns (total 10 columns):
 #   Column                      Non-Null Count  Dtype  
---  ------                      --------------  -----  
 0   object_id                   12000 non-null  int64  
 1   creation_time               12000 non-null  object 
 2   name                        12000 non-null  object 
 3   email                       12000 non-null  object 
 4   creation_source             12000 non-null  object 
 5   last_session_creation_time  8823 non-null   float64
 6   opted_in_to_mailing_list    12000 non-null  int64  
 7   enabled_for_marketing_drip  12000 non-null  int64  
 8   org_id                      12000 non-null  int64  
 9   invited_by_user_id          6417 non-null   float64
dtypes: float64(2), int64(4), object(4)
memory usage: 937.6+ KB
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 207917 entries, 0 to 207916
Data columns (total 3 columns):
 #   Column      Non-Null Count  

(None, None)

In [166]:
users.head(2)

,object_id,creation_time,name,email,creation_source,last_session_creation_time,opted_in_to_mailing_list,enabled_for_marketing_drip,org_id,invited_by_user_id
0,1,2014-04-22 03:53:30,Clausen August,AugustCClausen@yahoo.com,GUEST_INVITE,1.398139e+09,1,0,11,10803.0
1,2,2013-11-15 03:45:04,Poole Matthew,MatthewPoole@gustr.com,ORG_INVITE,1.396238e+09,0,0,1,316.0


In [167]:
engagement.head(20)

,time_stamp,user_id,visited
0,2014-04-22 03:53:30,1,1
1,2013-11-15 03:45:04,2,1
2,2013-11-29 03:45:04,2,1
3,2013-12-09 03:45:04,2,1
4,2013-12-25 03:45:04,2,1
5,2013-12-31 03:45:04,2,1
6,2014-01-08 03:45:04,2,1
7,2014-02-03 03:45:04,2,1
8,2014-02-08 03:45:04,2,1
9,2014-02-09 03:45:04,2,1


> Based on this engagement data, User id =2 should be tagged as a user who adopted. \
> Assert later that adopoion for user_id == 2 should be True

In [168]:
# Convert to timestams
users['creation_time'] = pd.to_datetime(users['creation_time'], format='%Y-%m-%d %H:%M:%S')
engagement['time_stamp'] = pd.to_datetime(engagement['time_stamp'], format='%Y-%m-%d %H:%M:%S')

In [169]:
# Rename column object_id
users = users.rename(columns={'object_id': 'user_id'})

In [ ]:
#Group by user id
eng_gby_user = engagement.groupby('user_id')

for k, v in list(eng_gby_user.groups.items())[:3]:
    print(f"User ID: {k}, Engagement Records: {v}")


User ID: 1, Engagement Records: Index([0], dtype='int64')
User ID: 2, Engagement Records: Index([1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14], dtype='int64')
User ID: 3, Engagement Records: Index([15], dtype='int64')


In [ ]:
# Sort by timestamp with each user
eng_by_user  = eng_gby_user.apply(lambda x: x.sort_values('time_stamp').reset_index(drop=True))

/tmp/ipykernel_5707/1248507798.py:1: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  eng_by_user  = eng_gby_user.apply(lambda x: x.sort_values('time_stamp').reset_index(drop=True))


In [178]:
# Use udf to check user adoption based on the given criteria

def is_adopted(data_df):

    # if there are less than 3 records, the user is not adopted    
    if len(data_df) < 3:
        return False

    # Sort and loop at the time stamps. Check if there are at least 3 records within 7 day period
    data_df = data_df.sort_values('time_stamp')
    for i in range(len(data_df) - 2):
        if data_df['time_stamp'].iloc[i + 2] - data_df['time_stamp'].iloc[i] < pd.Timedelta(days=7):
            return True

    return False

In [179]:
# Apply udf and check user adoption counts

eng_by_user_adopted  = eng_gby_user.apply(is_adopted).reset_index()
eng_by_user_adopted = eng_by_user_adopted.rename(columns={0: 'adopted'})
eng_by_user_adopted.adopted.value_counts()

/tmp/ipykernel_5707/432256658.py:3: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  eng_by_user_adopted  = eng_gby_user.apply(is_adopted).reset_index()


adopted
False    7221
True     1602
Name: count, dtype: int64

In [182]:
count_adopted_users = len(eng_by_user_adopted[eng_by_user_adopted.adopted == True])
print(f"Number of adopted users: {count_adopted_users}")
ratio_adopted_users = count_adopted_users / len(users)
print(f"Ratio of adopted users: {ratio_adopted_users:.2%}")

Number of adopted users: 1602
Ratio of adopted users: 13.35%


In [191]:
assert eng_by_user_adopted.loc[eng_by_user_adopted.user_id == 2]['adopted'].values[0] == True, "User 2 's label should be adopted=True. Check the logic."